# DBSCAN clustering

One distinct disadvantage of k-means clustering approaches is the requirement to know $k$ before actually clustering. Oftentimes, the number of clusters is not known beforehand and an extensive evaluation for all possible values of $k$ requires intensive computation times. The presented density-based DBSCAN method circumvents this issue.

In [ ]:
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

matplotlib.rc('image', cmap='nipy_spectral')
X = np.loadtxt('data/shapes.txt')

#### Task 1

Evaluate the k-means algorithm on the provided `shapes` data set and plot its results. To choose suitable values $k$, evaluate the average silhouette score for values for $k \in [2, 20]$. How would you rate the resulting cluster assignments?

In [ ]:
# Same function as defined in 8.1
def plot_clustering(data, labels, centroids=None, title='', ax=None):
    if data.shape[1] != 2:
        raise ValueError('Only two-dimensional data is supported.')
    if ax is None:
        _, ax = plt.subplots(1,1)
    ax.set_title(title)
    ax.scatter(data[:, 0], data[:, 1], c=labels)
    if centroids is not None:
        ax.scatter(centroids[:, 0], centroids[:, 1], c='magenta', s=150, marker='+')

In [ ]:
scores = []
# Your code below here
Ks = list(range(2,21))
for K in Ks:
    np.random.seed(42)
    kmeans = KMeans(n_clusters=K).fit(X)
    cluster_labels = kmeans.labels_
    score = silhouette_score(X, cluster_labels)
    scores.append(score)

plt.plot(Ks, scores)
plt.xlabel('K clusters')
plt.ylabel('Average silhouette score')
plt.title('Silhouette score for $K$ clusters')

# We see that 3 or 4 or maybe 6 cluster seem sensible according to silhouette score

good_Ks = [3, 4, 6]
for K in good_Ks:
    np.random.seed(42)  # ensure reproducible results
    kmeans = KMeans(n_clusters=K).fit(X)
    plot_clustering(X, kmeans.labels_, centroids=kmeans.cluster_centers_, title=f'$K={K}$, silhouette score: {scores[K-2]:1.3f}')

#### Solution

Judging which clustering makes most sense is not easy, e.g. should the dumbell-like shape on the right be one or two clusters? Generally, the results of k-means clustering seem not appropriate for this data set as blobs of data are often split and the spurious individual data points are all assigned to a specific cluster.

#### Task 2

a) Use the `DBSCAN` algorithm provided by sklearn to calculate a new clustering of the data. Plot the results in a scatter plot colored by cluster label. Which clustering result do you prefer, one of the k-means runs or the DBSCAN clustering?

b) Calculate the silhouette score for the DBSCAN clustering result, compare them and explain the difference. Which part of the silhouette score formula is presumably responsible for this difference?

*Note:* DBSCAN produces also an outlier cluster with the label `-1`. Using the `nipy_spectral` colormap, data points belonging to this cluster are always plotted in black. For calculation of the silhouette score, do not consider the samples contained in the outlier cluster.

In [ ]:
# Your code below here
dbscan = DBSCAN().fit(X)
select_idx = dbscan.labels_ >= 0
X_no_outliers = X[select_idx, :]
labels_no_outliers = dbscan.labels_[select_idx]
score = silhouette_score(X_no_outliers, labels_no_outliers)
plot_clustering(X, dbscan.labels_, title=f'DBSCAN, silhouette score: {score:1.3f}')

#### Solution

a) The DBSCAN algorithm produces clusters that make more sense, as connected data is put into one cluster. Spurious data points are considered outliers.

b) The DBSCAN silhouette score is lower than that of k-means. For this data set the average silhouette score of all data points is not a sensible metric to compare clustering results, as within cluster similarity being defined by average distance to cluster members is not supported by clusters that are two connected blobs. This is expressed as $a(i)$ in the formula for the silhouette score.

#### Task 3

Run the DBSCAN on the following modified versions of the shapes data set `X_1` and `X_2`. Plot the clustering results. Explain the results and try to modify DBSCAN's hyperparameters in such a way that it produces more sensible results for the modified datasets.

In [ ]:
X_1 = X*0.4
X_2 = X*2

# Your code below here

dbscan = DBSCAN().fit(X_1)
plot_clustering(X_1, dbscan.labels_, title=f'DBSCAN for X_1')

dbscan = DBSCAN().fit(X_2)
plot_clustering(X_2, dbscan.labels_, title=f'DBSCAN for X_2')

# as the 'new' data sets are only rescaled versions of the original data set,
# we simply need to adjust the density parameter eps

dbscan = DBSCAN(eps=.5*.4).fit(X_1)
plot_clustering(X_1, dbscan.labels_, title=f'Density corrected DBSCAN for X_1')

dbscan = DBSCAN(eps=.5*2).fit(X_2)
plot_clustering(X_2, dbscan.labels_, title=f'Density corrected DBSCAN for X_2')

#### Silhouette score for each data point individually

As we learned previously, the silhouette score can be calculated for each sample individually while we so far only used the average value.

The provided function `plot_silhouette_samples` visualizes the silhouette score for each data point individually. This function sorts the silhouette scores for each data point contained in a cluster in a descending fashion The red line indicates the average silhouette score of all data points across clusters. Note that this function discards outlier clusters, as these distort silhouette scores if considered as a regular cluster.

#### Task 4

Plot and interpret the results of using DBSCAN with default `eps=0.5` on data set `X` and compare it with silhouette scores obtained using k-means ($k=4$).

In [ ]:
from sklearn.metrics import silhouette_samples
from matplotlib import cm

def plot_silhouette_samples(data, labels, title=''):
    idx_no_outliers = labels >= 0
    fig, axes = plt.subplots(1, 2, figsize=(14,4))
    plot_clustering(data, labels, title='Clustering ' + title, ax=axes[0])
    # discard outliers for calculation of silhouette scores
    outliers_present = np.min(labels) == -1
    data = data[idx_no_outliers, :]
    labels = labels[idx_no_outliers]
    avg_score = silhouette_score(data, labels)
    ax = axes[1]
    ax.axvline(avg_score, linestyle='--', c='r')
    n_clusters = np.max(labels)
    sample_score = silhouette_samples(data, labels)
    y_lower = 10
    for i in range(n_clusters+1):
        ith_cluster_sample_score = \
            sample_score[labels == i]

        ith_cluster_sample_score.sort()

        size_cluster_i = ith_cluster_sample_score.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i+int(outliers_present)) /
                                 (n_clusters+int(outliers_present)))
        ax.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_sample_score,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 1  # 10 for the 0 samples
    ax.set_title('Silhouette samples ' + title)
    ax.set_xlim(-1, 1)

In [ ]:
# Your code below here
dbscan = DBSCAN().fit(X)
np.random.seed(42)
kmeans = KMeans(n_clusters=4).fit(X)
plot_silhouette_samples(X, dbscan.labels_, title='DBSCAN')
plot_silhouette_samples(X, kmeans.labels_, title='KMeans, $K=4$')

#### Solution

The small dense blobs of data are represented much better using the DBSCAN results. However especially the two clusters made up of connected blobs of data produce bad silhouette scores. The samples that produce negative low values for $s(i)$ can be accounted to the connected blobs. 
Intuitively put, the shapes of the data clusters in this data set do not fit well to the Euclidean distance metric used in the silhouette score. 

### Notes

Data sets in assignment 10 were taken from [1] and slightly modified.

[1] P. Fränti and S. Sieranoja
K-means properties on six clustering benchmark datasets
Applied Intelligence, 48 (12), 4743-4759, December 2018